# FEATURE ENGINEERING

This notebook logically follows the preprocessing step and performs feature engineering for modeling or further analysis.

✔ What it does:

- Loads the cleaned data from the previous step (pre-processed.csv).

Creates new features:
- Price Change %
- Target (binary label: will price rise next day?)
- Date features: weekday, month, quarter
- Rolling averages (MA_5, MA_10)
- Daily Return and log of Adjusted Close price


In [1]:
import os

In [2]:
import pandas as pd
import numpy as np

file_path = "data\PREPROCESSING\merged_stock_income.csv" # - ADAPT TO YOUR LOCAL ENVIRONMENT

df_merged = pd.read_csv(file_path)

df_merged.info()

<>:4: SyntaxWarning: invalid escape sequence '\P'
<>:4: SyntaxWarning: invalid escape sequence '\P'
C:\Users\SLO\AppData\Local\Temp\ipykernel_40972\3842545737.py:4: SyntaxWarning: invalid escape sequence '\P'
  file_path = "data\PREPROCESSING\merged_stock_income.csv" # - ADAPT TO YOUR LOCAL ENVIRONMENT


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18982067 entries, 0 to 18982066
Data columns (total 32 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Ticker                                    object 
 1   Date                                      object 
 2   SimFinId                                  int64  
 3   Open                                      float64
 4   High                                      float64
 5   Low                                       float64
 6   Close                                     float64
 7   Adj. Close                                float64
 8   Volume                                    int64  
 9   Shares Outstanding                        float64
 10  Currency                                  object 
 11  Fiscal Year                               int64  
 12  Fiscal Period                             object 
 13  Publish Date                              object 
 14  

In [3]:
df_merged.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common)
0,A,2019-04-26,45846,76.98,77.46,76.30,77.42,74.36,1608922,317515869.0,...,941000000.0,-22000000.0,-3.800000e+07,919000000.0,NaN,919000000.0,152000000.0,1071000000,1071000000,1071000000
1,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-2012488.0,-1937047.0,-1.949151e+06,-3949535.0,-4879.00,-3954414.0,120080400.0,-3954414,-3947435,-3947435
2,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-574174.0,-218833.0,-2.453920e+05,-793007.0,-973659.25,-793007.0,88160800.0,-793007,-774849,-774849
3,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-309402.0,41945.5,-8.240526e+07,-309402.0,-1942439.50,-309402.0,56241200.0,-309402,-298657,-298657
4,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-173991.0,302724.0,-1.645651e+08,128733.0,-2911219.75,128733.0,24321600.0,128733,94262,94262


New Columns: 

✅ Price Change % → Added daily price change % calculation (1 day lag)

✅ Target → Created Target column for market movement prediction

✅ Weekday (Weekday) → Categorical feature (Monday-Sunday) for classification models.

✅ Month (Month) → Categorical month number (1-12) for classification models.

✅ Quarter (Quarter) → Categorical feature (1-4) for classification models.

✅ Daily Returns (Daily_Return) → Percentage change in adjusted close price, useful for financial modeling.

✅ Moving Averages (MA_5, MA_10) → 5-day and 10-day moving averages for trend detection.

✅ Log Returns (Log_Returns) → Helps in normalizing price data for time series models.


In [4]:
df_merged["Price Change %"] = df_merged["Close"].pct_change() * 100  # Use correct column name

df_merged.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common),Price Change %
0,A,2019-04-26,45846,76.98,77.46,76.30,77.42,74.36,1608922,317515869.0,...,-22000000.0,-3.800000e+07,919000000.0,NaN,919000000.0,152000000.0,1071000000,1071000000,1071000000,NaN
1,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-1937047.0,-1.949151e+06,-3949535.0,-4879.00,-3954414.0,120080400.0,-3954414,-3947435,-3947435,-99.935417
2,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-218833.0,-2.453920e+05,-793007.0,-973659.25,-793007.0,88160800.0,-793007,-774849,-774849,0.000000
3,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,41945.5,-8.240526e+07,-309402.0,-1942439.50,-309402.0,56241200.0,-309402,-298657,-298657,0.000000
4,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,302724.0,-1.645651e+08,128733.0,-2911219.75,128733.0,24321600.0,128733,94262,94262,0.000000


Create the Target Variable for ML
To predict whether the price will go up or down, shift the adjusted close price:

In [5]:
df_merged["Target"] = (df_merged["Close"].shift(-1) > df_merged["Close"]).astype(int)  # 1 if price rises, 0 if falls
df_merged.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common),Price Change %,Target
0,A,2019-04-26,45846,76.98,77.46,76.30,77.42,74.36,1608922,317515869.0,...,-3.800000e+07,919000000.0,NaN,919000000.0,152000000.0,1071000000,1071000000,1071000000,NaN,0
1,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-1.949151e+06,-3949535.0,-4879.00,-3954414.0,120080400.0,-3954414,-3947435,-3947435,-99.935417,0
2,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-2.453920e+05,-793007.0,-973659.25,-793007.0,88160800.0,-793007,-774849,-774849,0.000000,0
3,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-8.240526e+07,-309402.0,-1942439.50,-309402.0,56241200.0,-309402,-298657,-298657,0.000000,0
4,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-1.645651e+08,128733.0,-2911219.75,128733.0,24321600.0,128733,94262,94262,0.000000,1


Handling Null Values: Since .pct_change() creates NaN values for the first row, fill them:

In [6]:
df_merged["Price Change %"].fillna(0, inplace=True)
df_merged["Target"].fillna(0, inplace=True)
df_merged.head()

C:\Users\SLO\AppData\Local\Temp\ipykernel_40972\287450061.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged["Price Change %"].fillna(0, inplace=True)
C:\Users\SLO\AppData\Local\Temp\ipykernel_40972\287450061.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common),Price Change %,Target
0,A,2019-04-26,45846,76.98,77.46,76.30,77.42,74.36,1608922,317515869.0,...,-3.800000e+07,919000000.0,NaN,919000000.0,152000000.0,1071000000,1071000000,1071000000,0.000000,0
1,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-1.949151e+06,-3949535.0,-4879.00,-3954414.0,120080400.0,-3954414,-3947435,-3947435,-99.935417,0
2,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-2.453920e+05,-793007.0,-973659.25,-793007.0,88160800.0,-793007,-774849,-774849,0.000000,0
3,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-8.240526e+07,-309402.0,-1942439.50,-309402.0,56241200.0,-309402,-298657,-298657,0.000000,0
4,VRDR,2019-04-26,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,...,-1.645651e+08,128733.0,-2911219.75,128733.0,24321600.0,128733,94262,94262,0.000000,1


In [7]:
# Set option to display all columns
pd.set_option('display.max_columns', None)

# Show DataFrame info
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18982067 entries, 0 to 18982066
Data columns (total 34 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Ticker                                    object 
 1   Date                                      object 
 2   SimFinId                                  int64  
 3   Open                                      float64
 4   High                                      float64
 5   Low                                       float64
 6   Close                                     float64
 7   Adj. Close                                float64
 8   Volume                                    int64  
 9   Shares Outstanding                        float64
 10  Currency                                  object 
 11  Fiscal Year                               int64  
 12  Fiscal Period                             object 
 13  Publish Date                              object 
 14  

In [8]:
df_merged.columns

Index(['Ticker', 'Date', 'SimFinId', 'Open', 'High', 'Low', 'Close',
       'Adj. Close', 'Volume', 'Shares Outstanding', 'Currency', 'Fiscal Year',
       'Fiscal Period', 'Publish Date', 'Restated Date', 'Shares (Basic)',
       'Shares (Diluted)', 'Revenue', 'Cost of Revenue', 'Gross Profit',
       'Operating Expenses', 'Selling, General & Administrative',
       'Operating Income (Loss)', 'Non-Operating Income (Loss)',
       'Interest Expense, Net', 'Pretax Income (Loss), Adj.',
       'Abnormal Gains (Losses)', 'Pretax Income (Loss)',
       'Income Tax (Expense) Benefit, Net',
       'Income (Loss) from Continuing Operations', 'Net Income',
       'Net Income (Common)', 'Price Change %', 'Target'],
      dtype='object')

Weekday column

In [9]:
df_merged['Date'] = pd.to_datetime(df_merged['Date']) # converting the "Date" column to a Datetime Object (double checking)

In [10]:
# 1. Weekday (Categorical for Classification)
df_merged['Weekday'] = df_merged['Date'].dt.day_name()  # 'Monday', 'Tuesday', etc.

Month Column

In [11]:
# 2. Month (Categorical for Classification)
df_merged['Month'] = df_merged['Date'].dt.month  # 1 (Jan) to 12 (Dec)

Quarter Column

In [12]:
# 3. Quarter (Categorical for Classification)
df_merged['Quarter'] = df_merged['Date'].dt.quarter  # 1 to 4

Daily Returns (Percentage Change)

In [13]:
df_merged['Daily_Return'] = df_merged['Close'].pct_change() * 100  # Convert to %

Moving Averages (5-day and 10-day)

In [14]:
df_merged['MA_5'] = df_merged['Close'].rolling(window=5, min_periods=1).mean()
df_merged['MA_10'] = df_merged['Close'].rolling(window=10, min_periods=1).mean()

Log Returns for Time Series Modeling

In [15]:
# Compute daily log-returns
df_merged['Log_Returns'] = np.log(df_merged['Close'] / df_merged['Close'].shift(1))

c:\Users\SLO\Documents\GitHub\python-group-project-2\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [30]:
test = df_merged[df_merged['Ticker'] == 'CRWD']
test[['Log_Returns', 'Close']].sample(5)

,Log_Returns,Close
Date,,
2022-09-12,4.440603,195.10
2020-04-13,0.000000,58.77
2023-07-18,0.000000,152.95
2019-09-05,0.000000,86.85
2023-11-08,1.817311,193.83


In [16]:
df_merged.set_index('Date', inplace=True)

In [17]:
df_merged.head()

,Ticker,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,Currency,Fiscal Year,Fiscal Period,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),Revenue,Cost of Revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common),Price Change %,Target,Weekday,Month,Quarter,Daily_Return,MA_5,MA_10,Log_Returns
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-04-26,A,45846,76.98,77.46,76.30,77.42,74.36,1608922,317515869.0,USD,2019,FY,2019-12-19,2021-12-17,314000000.0,318000000.0,5.163000e+09,-2.358000e+09,2.805000e+09,-1.864000e+09,-1.460000e+09,941000000.0,-22000000.0,-3.800000e+07,919000000.0,NaN,919000000.0,152000000.0,1071000000,1071000000,1071000000,0.000000,0,Friday,4,2,NaN,77.4200,77.4200,NaN
2019-04-26,VRDR,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,USD,2022,FY,2022-11-04,2023-10-16,812847811.0,812847811.0,3.816900e+04,-1.453560e+05,-1.071870e+05,-1.905301e+06,-1.905301e+06,-2012488.0,-1937047.0,-1.949151e+06,-3949535.0,-4879.00,-3954414.0,120080400.0,-3954414,-3947435,-3947435,-99.935417,0,Friday,4,2,-99.935417,38.7350,38.7350,-7.344977
2019-04-26,VRDR,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,USD,2021,FY,2021-11-12,2022-11-04,191278280.0,191278280.0,7.388640e+08,-4.410060e+08,3.570000e+08,-5.741740e+05,-5.741740e+05,-574174.0,-218833.0,-2.453920e+05,-793007.0,-973659.25,-793007.0,88160800.0,-793007,-774849,-774849,0.000000,0,Friday,4,2,0.000000,25.8400,25.8400,0.000000
2019-04-26,VRDR,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,USD,2020,FY,2020-10-22,2021-11-12,115082625.0,115082625.0,7.388640e+08,-4.410060e+08,3.570000e+08,-3.094020e+05,-3.094020e+05,-309402.0,41945.5,-8.240526e+07,-309402.0,-1942439.50,-309402.0,56241200.0,-309402,-298657,-298657,0.000000,0,Friday,4,2,0.000000,19.3925,19.3925,0.000000
2019-04-26,VRDR,12444417,0.05,0.05,0.05,0.05,0.05,1300,115038909.0,USD,2019,FY,2019-10-11,2020-10-22,115038909.0,115038909.0,1.203600e+04,-2.314100e+04,-1.110500e+04,-1.628860e+05,-1.628860e+05,-173991.0,302724.0,-1.645651e+08,128733.0,-2911219.75,128733.0,24321600.0,128733,94262,94262,0.000000,1,Friday,4,2,0.000000,15.5240,15.5240,0.000000


In [18]:
os.makedirs("data/ENRICH", exist_ok=True)

In [19]:
df_merged['Log_Returns'].dropna(inplace=True)

In [20]:
df_merged['Log_Returns'].describe()

c:\Users\SLO\Documents\GitHub\python-group-project-2\.venv\Lib\site-packages\numpy\_core\_methods.py:52: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


count    1.897229e+07
mean              NaN
std               NaN
min              -inf
25%     -1.212790e-01
50%      0.000000e+00
75%      1.215917e-01
max               inf
Name: Log_Returns, dtype: float64

In [21]:
df_merged['Close'].isna().sum()
df_merged['Close'].eq(0).sum()

np.int64(25287)

In [31]:
# Save enriched dataset
df_merged.to_csv("data/ENRICH/merged_stock_income.csv", index=True) 